# QR코드 읽어보기

파이썬에는 QR코드를 감지하고 디코딩 할 수 있는 많은 라이브러리들이 있습니다. 여기서는 몇몇가지의 라이브러리를 사용해서 QR코드를 디코딩 해보도록 하겠습니다.

## opencv 활용

> pip install opencv-python


파이썬에서 이미지 연산을 위해 많이 사용되는 opencv 라이브러리에도 QR코드를 디코딩 할 수 있는 함수가 내장 되어있습니다.

```detector = cv2.QRCodeDetector()```

opencv 의 QRCodeDetector() 라이브러리를 사용해서 QR코드를 디코딩 할 수 있는 객체를 생성합니다.

```data, box, straight_qrcode = detector.detectAndDecode('QR코드 이미지')```

detectAndDecode() 함수는 QR코드 데이터, QR코드 영역의 Rect, QR코드 이미지, 3가지의 값을 리턴합니다.

In [ ]:
import cv2

def read_qrcode_cv2(open_cv_image):
    detector = cv2.QRCodeDetector()
    data, bbox, _ = detector.detectAndDecode(open_cv_image)
    if data:
        print('QR코드 데이터: {}'.format(data))
        print('QR코드 위치: {}'.format(bbox))

        lefttop = tuple(bbox[0][0])
        rightbottom = tuple(bbox[0][2])
        cv2.rectangle(open_cv_image, lefttop, rightbottom, (0, 0, 255), 5)

        cv2.imshow('img', open_cv_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("인식 오류!")

filepath1 = "qrcode_1.jpg"
filepath2 = "qrcode_2.jpg"
cv_image1 = cv2.imread(filepath1)
cv_image2 = cv2.imread(filepath2)
read_qrcode_cv2(cv_image1)
read_qrcode_cv2(cv_image2)

위의 코드에서 작성한 ```read_qrcode_cv2``` 함수는 ```cv2.QRCodeDetector()``` 라이브러리를 사용해서 이미지의 QR코드를 인식하는 내용입니다. 그런데 막상 위의 코드를 실행해보면 샘플로 사용한 ```qrcode_1.jpg``` 는 인식을 잘하는 반면에 ```qrcode_2.jpg``` 파일은 인식하지 못합니다. 여러 이미지를 테스트 해보면 ```cv2``` 라이브러리의 인식률은 조금 아쉬운 부분이 있습니다.

## pyzbar 활용

pyzbar는 QR코드 디텍터 라이브러리에서 크로스플랫폼을 지원하고 속도와 인식률이 높은 파이썬용 zbar 라이브러리 입니다. <font color="red" style="background-color:#eeeeee;">pyzbar를 사용하기 위해선 먼저 zbar 라이브러리를 설치 해야하는데 각 운영체제마다 설치하는 방법이 다릅니다.</font> 윈도우의 경우 ImportError 가 발생하는 경우에는 [Visual C++ 2013 재배포패키지](https://www.microsoft.com/en-US/download/details.aspx?id=40784)를 반드시 설치 해야 합니다. 다른 운영체제의 설치 방법은 [링크](http://zbar.sourceforge.net/download.html) 를 통해 확인 하실 수 있습니다.

```pip install pyzbar```

zbar를 인스톨 했으면 파이썬에서 사용하기 위해 pyzbar 라이브러리를 설치해야 합니다.

In [ ]:
import pyzbar.pyzbar as pyzbar
import cv2

def read_qrcode_zbar(open_cv_image):
    gray = cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2GRAY)
    decoded = pyzbar.decode(gray)
    for d in decoded:
        x, y, w, h = d.rect

        barcode_data = d.data.decode("utf-8")
        barcode_type = d.type

        cv2.rectangle(open_cv_image, (x, y), (x + w, y + h), (0, 0, 255), 2)

        print(barcode_data, barcode_type)
        text = '%s (%s)' % (barcode_data, barcode_type)
        cv2.putText(open_cv_image, text, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 1, cv2.LINE_AA)

    print(decoded)
    cv2.imshow('img', open_cv_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
filepath1 = "qrcode_1.jpg"
filepath2 = "qrcode_2.jpg"
cv_image1 = cv2.imread(filepath1)
cv_image2 = cv2.imread(filepath2)
read_qrcode_zbar(cv_image1)
read_qrcode_zbar(cv_image2)

pyzbar 는 내부적으로 1채널 흑백이미지를 사용하기때문에 인자로 넘어온 이미지를 그레이 이미지로 컨버트 시켜줘야합니다. 한장의 이미지에서도 여러개의 QR코드를 감지할 수 있기 때문에 decode() 함수의 결과를 리스트로 반환 됩니다. decode() 함수는 [ {rect:QR코드위치, data:QR코드데이터} ] 의 dict 형태가 리스트로 넘어옵니다. 또한 ```cv2``` 와 다르게 샘플로 제공한 ```qrcode_1.jpg``` 와 ```qrcode_2.jpg``` 파일 모두 문제없이 잘 인식 됩니다.

## 웹캠을 통해 QR코드 인식하고 저장하기

In [ ]:
import pyzbar.pyzbar as pyzbar
import cv2

cap = cv2.VideoCapture(0)

i = 0
while(cap.isOpened()):
    ret, img = cap.read()
    if not ret:
        continue
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    decoded = pyzbar.decode(gray)

    for d in decoded: 
        x, y, w, h = d.rect

        barcode_data = d.data.decode("utf-8")
        barcode_type = d.type

        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)

        text = '%s (%s)' % (barcode_data, barcode_type)
        cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('img', img)

    key = cv2.waitKey(1)
    
    # q 누르면 종료 s 누르면 현재 이미지 저장
    if key == ord('q'):
        break
    elif key == ord('s'):
        i += 1
        cv2.imwrite('c_%03d.jpg' % i, img)

cap.release()
cv2.destroyAllWindows()